In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local').config('spark.driver.memory', '8g').appName('Ready_for_analysis').getOrCreate()



In [4]:
import psycopg2
import pandas as pd
import numpy as np

def connect_redshift_db():
    cnx = psycopg2.connect(dbname= 'dwh',
                     host='redshift-prod.data.service.home24.net',
                     port= '5439',
                     user= 'gabriele_sabato',
                     password= 'zQZ8x6ZhRoTf7GS')
    return cnx
cnx_redshift = connect_redshift_db()
cursor_redshift = cnx_redshift.cursor()

In [5]:
def query_to_dataframe(_cursor, _query):
    _cursor.execute(_query)
    colnames = [desc[0] for desc in _cursor.description]
    print(colnames)
    return pd.DataFrame(_cursor.fetchall(), columns=colnames)

In [6]:
#CREATE MARKET SESSION SPARK DF
sql_query_market_session = """
SELECT report_date, sum(sessions) as session
from public.marketing_daily_dashboard_base
WHERE report_date >= '2020-01-01'
group by 1
order by 1
"""

df_marketing_session = query_to_dataframe(cursor_redshift,sql_query_market_session)

df_spark_market_session = spark.createDataFrame(df_marketing_session)

['report_date', 'session']


In [7]:
#CREATE MARKET SESSION SPARK DF
sql_query_market_cost = """
SELECT report_date, sum(cost) as marketing_cost_spend
FROM public.marketing_cost_consolidated
WHERE report_date >= '2020-01-01'
AND marketing_channel != 'Sales personnel'
group by 1
order by 1
"""

df_marketing_cost = query_to_dataframe(cursor_redshift,sql_query_market_cost)

df_spark_market_cost = spark.createDataFrame(df_marketing_cost)



['report_date', 'marketing_cost_spend']


In [8]:
df = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/price_elasticity_model_data/part-*',
    header=True)

In [9]:
df.createOrReplaceTempView('model_data_table')


In [10]:
#df.show()

In [11]:
df_d_item = spark.read.parquet('/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/d_item/*')


In [12]:
df_d_item.createOrReplaceTempView('d_item_tmp')

In [13]:
df_item_visits = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/item_visits/*2020*/*.parquet',header=True)

In [14]:
df_f_orders = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/f_orders/*.parquet',header=True)

df_f_orders.createOrReplaceTempView('f_orders')

In [15]:
df_d_calendar = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/d_calendar/*.parquet',header=True)

df_d_calendar.createOrReplaceTempView('d_calendar')



In [16]:
df_d_order_flags = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/d_order_flags/*.parquet',header=True)

df_d_order_flags.createOrReplaceTempView('d_order_flags')



In [17]:
df_list_top_seller = spark.sql("""
SELECT di.item_code,
       sum(fo.amount + amount_discount) AS order_amount

  FROM f_orders as fo
       INNER JOIN d_calendar c ON c.date_skey = fo.order_date_skey
       INNER JOIN d_item_tmp di ON di.item_skey = fo.item_skey
INNER JOIN d_order_flags dof on dof.order_flags_skey = fo.order_flags_skey

 WHERE c.date >= '2020-01-01' --current_date - 1*interval ' 1 year'
and dof.is_cancelled_immediately = 'is not cancelled immediately'
and dof.is_cancelled_after_payment = 'is not cancelled after payment'
and dof.is_cancelled_before_payment = 'is not cancelled before payment'
group by di.item_code
ORDER BY order_amount DESC""")

In [18]:
df_list_top_seller.createOrReplaceTempView('top_seller_list')

#df_list_top_seller.show()



In [22]:
df_list_top_seller_10K= spark.sql("""
SELECT item_code
from top_seller_list
LIMIT 10000""")



Py4JJavaError: An error occurred while calling o297.showString.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange hashpartitioning(item_code#19, 200), true, [id=#301]
+- *(4) HashAggregate(keys=[item_code#19], functions=[partial_sum(CheckOverflow((promote_precision(cast(amount#153 as decimal(38,17))) + promote_precision(cast(amount_discount#155 as decimal(38,17)))), DecimalType(38,17), true))], output=[item_code#19, sum#249])
   +- *(4) Project [amount#153, amount_discount#155, item_code#19]
      +- *(4) BroadcastHashJoin [cast(order_flags_skey#138 as int)], [order_flags_skey#232], Inner, BuildRight
         :- *(4) Project [order_flags_skey#138, amount#153, amount_discount#155, item_code#19]
         :  +- *(4) BroadcastHashJoin [item_skey#134L], [item_skey#18L], Inner, BuildRight
         :     :- *(4) Project [item_skey#134L, order_flags_skey#138, amount#153, amount_discount#155]
         :     :  +- *(4) BroadcastHashJoin [order_date_skey#13

In [ ]:
top_seller_list_string_10k = df_list_top_seller_10K.rdd.map(lambda x: x.item_code).collect()


In [ ]:
import numpy as np

List_of_n_items = np.arange(10,101,10)
print(List_of_n_items)

map_of_items = {}

#DEFINE Number of items and get list
def get_list_of_items(N_items = 10,top_seller_list = top_seller_list_string_10k):
    selected_items = top_seller_list[:N_items]
    selected_items[0]= """'""" + selected_items[0]
    selected_items[-1]= selected_items[-1] + """'"""
    separator =  """' , '"""
    f_string = separator.join(selected_items)
    #print(f_string)
    return f_string

for n_item in List_of_n_items:
    map_of_items[n_item]=get_list_of_items(n_item)

print(map_of_items[10])

In [ ]:
DF_list = []
#for keys in map_of_items.keys():
#    print(map_of_items[keys])
print(map_of_items[10])
final_string = map_of_items[10]

In [30]:
df_item_visits.createOrReplaceTempView('item_visits')

df_item_visits_df = spark.sql("""SELECT item_code,
                                         meta_date,
                                        SUM(unique_views) as all_unique_views
                                    from item_visits
                                  where item_code in ({lista})
                                  GROUP BY item_code, meta_date""".format(lista=final_string))
df_item_visits_df.show()

+------------------+----------+----------------+
|         item_code| meta_date|all_unique_views|
+------------------+----------+----------------+
|000000001000021540|2020-10-24|             316|
|000000001000113432|2020-10-20|             433|
|000000001000054983|2020-01-19|             295|
|000000001000054984|2020-05-23|             431|
|000000001000054984|2020-01-28|             822|
|000000001000015496|2020-03-08|             670|
|000000001000113432|2020-03-30|             298|
|000000001000062031|2020-03-07|             334|
|000000001000054983|2020-06-09|             123|
|000000001000054983|2020-12-19|             252|
|000000001000113432|2020-07-01|             218|
|000000001000113432|2020-03-20|             183|
|000000001000021718|2020-06-27|             315|
|000000001000113432|2020-08-29|             307|
|000000001000021718|2020-08-29|             530|
|000000001000021718|2020-11-18|             732|
|000000001000021540|2020-01-08|             491|
|000000001000054984|

In [31]:
df2 = spark.sql("""SELECT * from model_data_table where update_date >= '2020-01-01' and item_code in ({lista}) ORDER BY update_date""".format(lista=final_string))
df2.show()

+------------------+-----------+--------------+------------+-----+
|         item_code|update_date|delivery_weeks|  item_price|sales|
+------------------+-----------+--------------+------------+-----+
|000000001000021540| 2020-01-01|             2|     1449.99|    3|
|000000001000015496| 2020-01-01|             2|      699.99|    2|
|000000001000113432| 2020-01-01|             2|      849.99|    1|
|000000001000062031| 2020-01-01|             2|     1899.99|    0|
|000000001000016021| 2020-01-01|             5|119.98999999|   21|
|000000001000021539| 2020-01-01|             2|     1449.99|    1|
|000000001000016133| 2020-01-01|             1|119.98999999|    3|
|000000001000054984| 2020-01-01|             2|      749.99|    0|
|000000001000021718| 2020-01-01|             4|     1049.99|    2|
|000000001000054983| 2020-01-01|             2|      749.99|    0|
|000000001000062031| 2020-01-02|             2|     1899.99|    0|
|000000001000054983| 2020-01-02|             2|      749.99|  

In [32]:
df4 = spark.sql ("""SELECT item_code, item_main_category, item_sub_category_1, item_sub_category_2,item_parent_item_code from d_item_tmp where item_code in ({lista})""".format(lista=final_string))
df4.show()

+------------------+--------------------+-------------------+-------------------+---------------------+
|         item_code|  item_main_category|item_sub_category_1|item_sub_category_2|item_parent_item_code|
+------------------+--------------------+-------------------+-------------------+---------------------+
|000000001000015496|             POLSTER|        EINZELSOFAS|        EINZELSOFAS|   000000008000001000|
|000000001000016021|ESSZIMMERSTÜHLE&B...|    ESSZIMMERSTÜHLE|          KLASSISCH|   000000008000001083|
|000000001000016133|ESSZIMMERSTÜHLE&B...|    ESSZIMMERSTÜHLE|          KLASSISCH|   000000008000001083|
|000000001000021539|             POLSTER|   WOHNLANDSCHAFTEN|   WOHNLANDSCHAFTEN|   000000008000001826|
|000000001000021540|             POLSTER|   WOHNLANDSCHAFTEN|   WOHNLANDSCHAFTEN|   000000008000001826|
|000000001000021718|             POLSTER|           ECKSOFAS|          LONGCHAIR|   000000008000001863|
|000000001000054983|             POLSTER|           ECKSOFAS|   

In [33]:
df4.createOrReplaceTempView('d_item_filtered')
df2.createOrReplaceTempView('model_data_table_YTD')
df_item_visits_df.createOrReplaceTempView('item_views_YTD')
df_spark_market_session.createOrReplaceTempView('market_session')
df_spark_market_cost.createOrReplaceTempView('market_cost')

In [34]:
df_item_visits_df.describe(['item_code']).show()

+-------+------------------+
|summary|         item_code|
+-------+------------------+
|  count|              3660|
|   mean|    1.0000397877E9|
| stddev|30148.881723687216|
|    min|000000001000015496|
|    max|000000001000113432|
+-------+------------------+



In [37]:
sql_price_visit_join= """
SELECT mdt_YTD.*, it_YTD.all_unique_views, ms.session, mc.marketing_cost_spend
FROM model_data_table_YTD as mdt_YTD
LEFT JOIN item_views_YTD as it_YTD
on mdt_YTD.item_code = it_YTD.item_code and mdt_YTD.update_date = it_YTD.meta_date
LEFT JOIN market_session as ms
on mdt_YTD.update_date = ms.report_date
LEFT JOIN market_cost as mc
on mdt_YTD.update_date = mc.report_date
"""

In [45]:
df_price_visit_join = spark.sql(sql_price_visit_join)
df_price_visit_join.show()

+------------------+-----------+--------------+------------+-----+----------------+---------+--------------------+
|         item_code|update_date|delivery_weeks|  item_price|sales|all_unique_views|  session|marketing_cost_spend|
+------------------+-----------+--------------+------------+-----+----------------+---------+--------------------+
|000000001000015496| 2020-08-24|             2|      699.99|    3|             364|107733918|181069.3913969999...|
|000000001000054983| 2020-08-24|             2|      749.99|    5|             177|107733918|181069.3913969999...|
|000000001000113432| 2020-08-24|             2|      849.99|    2|             293|107733918|181069.3913969999...|
|000000001000062031| 2020-08-24|             4|     1549.99|    2|             344|107733918|181069.3913969999...|
|000000001000021718| 2020-08-24|             8|      829.99|    0|             418|107733918|181069.3913969999...|
|000000001000016021| 2020-08-24|             1|119.98999999|   16|             3

In [46]:
df_price_visit_join.createOrReplaceTempView('mdt_YTD_it')

#df_price_visit_join.describe(['item_code']).show()
df_price_visit_join.show()



+------------------+-----------+--------------+------------+-----+----------------+---------+--------------------+
|         item_code|update_date|delivery_weeks|  item_price|sales|all_unique_views|  session|marketing_cost_spend|
+------------------+-----------+--------------+------------+-----+----------------+---------+--------------------+
|000000001000015496| 2020-08-24|             2|      699.99|    3|             364|107733918|181069.3913969999...|
|000000001000054983| 2020-08-24|             2|      749.99|    5|             177|107733918|181069.3913969999...|
|000000001000113432| 2020-08-24|             2|      849.99|    2|             293|107733918|181069.3913969999...|
|000000001000062031| 2020-08-24|             4|     1549.99|    2|             344|107733918|181069.3913969999...|
|000000001000021718| 2020-08-24|             8|      829.99|    0|             418|107733918|181069.3913969999...|
|000000001000016021| 2020-08-24|             1|119.98999999|   16|             3

In [47]:
sql_top_item_query_group_by = """
WITH tmp_tbl AS (
      SELECT *,
             CASE
                 WHEN lag(delivery_weeks, 1) OVER (PARTITION BY item_code ORDER BY update_date ASC) =
                      delivery_weeks
                     AND
                      lag(item_price, 1) OVER (PARTITION BY item_code ORDER BY update_date ASC) =
                      item_price
                     THEN NULL
                 ELSE RANK() OVER (PARTITION BY item_code ORDER BY update_date)
                 END AS ranking_col --filled with row number or delivery week if the previous one is part of the same group,
        FROM mdt_YTD_it
  ),
       tmp_tbl2 AS (
           SELECT update_date,
                  item_code,
                  sales,
                  all_unique_views,
                  item_price,
                  delivery_weeks,
                  session,
                  marketing_cost_spend,
                  CASE
                      WHEN ranking_col IS NULL
                          THEN last(ranking_col, True) OVER (PARTITION BY item_code ORDER BY update_date ROWS BETWEEN UNBOUNDED PRECEDING and 1 PRECEDING)
                      ELSE ranking_col
                      END AS ranks
             FROM tmp_tbl
       )
SELECT CAST(MIN(update_date) as date)                     AS min_date,
       CAST (MAX(update_date) as date)                    AS max_date,
       item_code,
       item_price,
       delivery_weeks,
       avg(sales) as avg_sales,
       avg(all_unique_views) as avg_unique_views,
       avg(session) as avg_sessions,
       avg(marketing_cost_spend) as avg_market_cost_spend,
       CASE WHEN sum(all_unique_views) <> 0 THEN sum(sales)/sum(all_unique_views) ELSE NULL END as conv_rate,
       CAST (datediff(MAX(update_date), MIN(update_date) ) + 1 as int) AS bin
  FROM tmp_tbl2
 GROUP BY ranks, delivery_weeks, item_price, item_code
 ORDER BY item_code, min_date ASC;
"""

In [48]:
sql_top_item_df = spark.sql(sql_top_item_query_group_by)

In [49]:
sql_top_item_df.show(5000,False)

+----------+----------+------------------+------------+--------------+-------------------+------------------+--------------------+-----------------------------+---------------------+---+
|min_date  |max_date  |item_code         |item_price  |delivery_weeks|avg_sales          |avg_unique_views  |avg_sessions        |avg_market_cost_spend        |conv_rate            |bin|
+----------+----------+------------------+------------+--------------+-------------------+------------------+--------------------+-----------------------------+---------------------+---+
|2020-01-01|2020-01-07|000000001000015496|699.99      |2             |2.857142857142857  |565.2857142857143 |1.6069389757142857E8|222777.8834131428562291500000|0.005054334091483447 |7  |
|2020-01-08|2020-02-05|000000001000015496|659.99      |2             |1.1724137931034482 |450.86206896551727|1.5555189872413793E8|225651.6244066206897265490000|0.00260038240917782  |29 |
|2020-02-06|2020-03-07|000000001000015496|699.99      |2         

In [50]:
#CREATE A TABLE VIEW FOR Top10 sellers
sql_top_item_df.createOrReplaceTempView('top_sellers')


In [51]:
#JOIN THE TWO TABLEs
sql_join_query = " SELECT top.*, di.item_parent_item_code, di.item_main_category, di.item_sub_category_1, di.item_sub_category_2" \
                 "        from top_sellers as top " \
                 "INNER JOIN d_item_filtered as di on di.item_code = top.item_code "\
                 "ORDER BY top.min_date ASC"

In [52]:
print(sql_join_query)


 SELECT top.*, di.item_parent_item_code, di.item_main_category, di.item_sub_category_1, di.item_sub_category_2        from top_sellers as top INNER JOIN d_item_filtered as di on di.item_code = top.item_code ORDER BY top.min_date ASC


In [53]:
final_top_seller_df = spark.sql(sql_join_query)

In [54]:
final_top_seller_df.show(3000,False)


+----------+----------+------------------+------------+--------------+-------------------+------------------+--------------------+-----------------------------+---------------------+---+---------------------+---------------------+-------------------+-------------------+
|min_date  |max_date  |item_code         |item_price  |delivery_weeks|avg_sales          |avg_unique_views  |avg_sessions        |avg_market_cost_spend        |conv_rate            |bin|item_parent_item_code|item_main_category   |item_sub_category_1|item_sub_category_2|
+----------+----------+------------------+------------+--------------+-------------------+------------------+--------------------+-----------------------------+---------------------+---+---------------------+---------------------+-------------------+-------------------+
|2020-01-01|2020-01-02|000000001000021540|1449.99     |2             |1.5                |323.5             |1.38280645E8        |178888.5803034999980398150000|0.00463678516228748  |2  |0

In [120]:
pd_df_top_sellers = final_top_seller_df.toPandas()


In [121]:
import pickle
import pandas as pd

In [122]:
pd_df_top_sellers['log_price'] = np.log(pd_df_top_sellers['item_price']+0.0001)

In [123]:
pd_df_top_sellers['log_delivery_weeks'] = np.log(pd_df_top_sellers['delivery_weeks']+0.0001)

In [124]:
pd_df_top_sellers['log_sales'] = np.log(pd_df_top_sellers['avg_sales']+0.0001)


In [125]:
pd_df_top_sellers['log_avg_unique_views'] = np.log(pd_df_top_sellers['avg_unique_views']+0.0001)


In [126]:
pd_df_top_sellers['log_conv_rate']= np.log(pd_df_top_sellers['conv_rate']+0.0001)

In [127]:
pd_df_top_sellers.to_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/Top10old_seller_YTD_group_by_item_visit.pickle')